In [2]:
import pyaudiowpatch as pyaudio
import numpy as np

In [3]:
RATE = 16000
CHUNK = 256

pyaud = pyaudio.PyAudio()
wasapi_info = pyaud.get_host_api_info_by_type(pyaudio.paWASAPI)
default_speakers = pyaud.get_device_info_by_index(wasapi_info["defaultOutputDevice"])

if not default_speakers["isLoopbackDevice"]:
    for loopback in pyaud.get_loopback_device_info_generator():
        """
        Try to find loopback device with same name(and [Loopback suffix]).
        Unfortunately, this is the most adequate way at the moment.
        """
        if default_speakers["name"] in loopback["name"]:
            default_speakers = loopback
            break
    else:
        exit()
        
print(f"Recording from: ({default_speakers['index']}){default_speakers['name']}")

stream = pyaud.open(format=pyaudio.paInt16, channels=default_speakers["maxInputChannels"], rate=int(default_speakers["defaultSampleRate"]), input=True, input_device_index=default_speakers["index"], frames_per_buffer=CHUNK)



Recording from: (17)Speakers (PRO X Wireless Gaming Headset) [Loopback]


In [4]:
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [6]:
frames = []
for i in range(int(RATE/CHUNK * 2)):
    data = stream.read(CHUNK)
    frames.append(np.frombuffer(data, dtype=np.int16))

# stream.stop_stream()
# stream.close()
# pyaud.terminate()

print('reached')

audio_data = np.concatenate(frames)

frequencies, times, Sxx = spectrogram(audio_data, fs=RATE, nperseg=CHUNK)
plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram')
plt.colorbar(label='Power/Frequency [dB/Hz]')
plt.show()

OSError: [Errno -9988] Stream closed